# Background

Deep-Learning Package Zoo
* Torch
* Caffe
* Theano (Keras, Lasagne)
* CuDNN
* Tensorflow 
* Mxnet
* etc.

### package design choices

* Model specification: Configuration file (e.g. Caffe, DistBelief, CNTK) 
* programmatic generation (e.g. Torch, Theano, Tensorflow)
    * choice of high-level language: Lua (Torch) vs. Python (Theano, Tensorflow) vs others.
    * python has rich community and library infrastructure.

### Theano VS TensorFlow

* Theano and TensorFlow are very similar systems
* TensorFlow has better support for distributed systems though, and has development funded by Google
* Theano is an academic project.

### Tensor Definition

In [1]:
from IPython.display import Image

![tensor_def](figs/tensor_def.png)

### tensorflow vs numpy

* TensorFlow and Numpy are quite similar. Both are N-d array libraries!
* Numpy has Ndarray support, but doesn’t offer methods to create tensor functions and automatically compute derivatives (+ no GPU support).

![np_vs_tf](figs/np_vs_tf.png)

### Exe01

In [2]:
import tensorflow as tf

In [3]:
# this is just a computation graph
ta = tf.zeros((2,2))
print(ta)

Tensor("zeros:0", shape=(2, 2), dtype=float32)


In [4]:
init = tf.global_variables_initializer()   # create nodes in the graph
sess = tf.Session() 
sess.run(init)  # initialize all the variables

v = sess.run(ta)  
print(v)

sess.close()  # close the session 

[[ 0.  0.]
 [ 0.  0.]]


### Some background knowledge

* TensorFlow programs are usually structured into a construction phase, that assembles a graph, and an execution phase that uses a session to execute ops in the graph.

* A Session object encapsulates the environment in which Tensor objects are evaluated.

* All computations add nodes to global default graph


### more elegant implementation

In [5]:
ta = tf.zeros((2,2))
x = tf.Variable(3, name="x") # variable objects can be initialized from constants or random values
y = tf.Variable('hello', name="y")
z = tf.constant(5.0)
R = tf.Variable(tf.random_normal((2,2)), name="random_weights")  

init = tf.global_variables_initializer()   # create nodes in the graph
with tf.Session() as sess:
    init.run()
    print(sess.run(ta))
    print(sess.run(x))
    print(sess.run(y))
    print(sess.run(z))

[[ 0.  0.]
 [ 0.  0.]]
3
b'hello'
5.0


### Why I should reset graph?

In Jupyter (or in a Python shell), it is common to run the same commands more than once while
you are experimenting. As a result, *** you may end up with a default graph containing many duplicate
nodes ***. One solution is to restart the Jupyter kernel (or the Python shell), but a more convenient
solution is to just reset the default graph by running tf.reset_default_graph().


## Exe02

In [6]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    init.run()
    print(y.eval())  # 10
    print(z.eval())  # 15
    #print(sess.run(y))
    #print(sess.run(z))

10
15


In [7]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)  # 10
    print(z_val)  # 15

10
15


### exe03

In [8]:
state = tf.Variable(0, name="counter")
#new_value = state + 1
new_value = tf.add(state, tf.constant(1)) #  new_value = state + 1
update = tf.assign(state, new_value)  #  state = new_value

init = tf.global_variables_initializer()   # create nodes in the graph
with tf.Session() as sess:
    init.run()
    print(sess.run(state))
    for i in range(3):
        sess.run(update)
        print(sess.run(state))

0
1
2
3


### input numpy array to tf

In [9]:
import numpy as np

a = np.zeros((3,3))
ta = tf.convert_to_tensor(a)

init = tf.global_variables_initializer()   # create nodes in the graph
with tf.Session() as sess:
    init.run()
    print(sess.run(ta))

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


### placeholder and feed dictionaries

* Inputting data with tf.convert_to_tensor() is convenient, but doesn’t scale.
* Use tf.placeholder variables (dummy nodes that provide entry points for data to computational graph).
* A feed_dict is a python dictionary mapping from tf. placeholder vars (or their names) to data (numpy arrays, lists, etc.).
 

In [10]:
input1 = tf.placeholder(tf.float32)
input2 = tf.placeholder(tf.float32)

#output = tf.mul(input1, input2) # deprecated!
output = tf.multiply(input1, input2)


init = tf.global_variables_initializer()   # create nodes in the graph
with tf.Session() as sess:
    init.run()
    print(sess.run([output], feed_dict={input1:[7.], input2:[2.]}))  #   7 x 2 = 14

[array([ 14.], dtype=float32)]


# References